In [1]:
import torch
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, device_map="auto"
)
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

c:\Users\Pavel\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pavel\.cache\huggingface\hub\models--deepseek-ai--deepseek-math-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

In [26]:
def get_answer_from_model(task_text):
    sys_prompt = """
    You are a mathematician. You need to solve the task.
    """
    messages = [
        {
            "role": "system",
            "content": sys_prompt,
        },
        {
            "role": "user",
            "content": f"Answer only the final numeric result, nothing else. Task: {task_text}",
        }
    ]
    input_tensor = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    )
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=50)
    result = tokenizer.decode(
        outputs[0][input_tensor.shape[1] :], skip_special_tokens=True
    )

    cleaned = re.findall(r"-?\d+(?:\.\d+)?", result)
    if cleaned:
        extracted_answer = cleaned[0]
    else:
        extracted_answer = result.strip()
    return extracted_answer

In [27]:
df = pd.read_csv("/kaggle/input/aiijc-llm-teacher/train.csv")

submission_rows = []
for idx, row in df.iterrows():
    task = row["task"]
    answer = get_answer_from_model(task)
    submission_rows.append({"task": task, "answer": f"[{answer}]"})

submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv("submission.csv", index=False)